In [1]:
!pip install gensim

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple


In [2]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import gc

In [3]:
ad=pd.read_csv('data/train_preliminary/ad.csv')
click_log=pd.read_csv('data/train_preliminary/click_log.csv')

In [4]:
result = pd.merge(click_log,ad, on='creative_id')
result.drop(['creative_id','product_id','product_category','advertiser_id','industry'],axis=1,inplace=True)
train_ad=result

del ad,click_log,result
gc.collect()

0

In [5]:
ad=pd.read_csv('data/test/ad.csv')
click_log=pd.read_csv('data/test/click_log.csv')

In [6]:
result = pd.merge(click_log,ad, on='creative_id')
result.drop(['creative_id','product_id','product_category','advertiser_id','industry'],axis=1,inplace=True)
test_ad=result

del ad,click_log,result
gc.collect()

0

In [7]:
train_test_ad = train_ad.append(test_ad)

del train_ad,test_ad
gc.collect()

0

In [8]:
train_test_ad

,time,user_id,click_times,ad_id
0,9,30920,1,504423
1,15,320815,1,504423
2,11,355089,1,504423
3,9,363442,1,504423
4,14,370513,1,504423
...,...,...,...,...
33585507,49,3131983,1,1757820
33585508,4,3131984,1,168395
33585509,5,3131984,1,290114
33585510,16,3131984,1,793805


In [9]:
train_test_ad = train_test_ad.sort_values(by=['user_id', 'time'])
train_test_ad

,time,user_id,click_times,ad_id
16203201,20,1,1,773445
23067445,20,1,1,188507
26781023,20,1,1,724607
22502265,39,1,1,1458878
9498187,40,1,1,109959
...,...,...,...,...
18977219,75,4000000,1,3096233
27901133,75,4000000,1,3135640
9687462,76,4000000,1,331268
17287071,76,4000000,1,2868147


In [10]:
train_test_id = train_test_ad[['user_id']].drop_duplicates()
train_test_id

,user_id
16203201,1
6321891,2
11129786,3
24566977,4
22548585,5
...,...
24877162,3999996
25310722,3999997
22557680,3999998
11919336,3999999


In [11]:
train_test_features = train_test_ad[['user_id', 'ad_id']]
train_test_features

,user_id,ad_id
16203201,1,773445
23067445,1,188507
26781023,1,724607
22502265,1,1458878
9498187,1,109959
...,...,...
18977219,4000000,3096233
27901133,4000000,3135640
9687462,4000000,331268
17287071,4000000,2868147


In [12]:
#造句
def get_sentences(df_, key, name):
    sentences = df_.groupby([key]).apply(lambda x: x[name].tolist()).tolist()
    sentences3 = []
    for i in range(len(sentences)):
        sentences2 = list(map(str, sentences[i]))
        sentences3.append(sentences2)
    return sentences3

sentences = get_sentences(train_test_features, 'user_id', 'ad_id')
len(sentences)

1900000

In [13]:
#w2c
def get_embedding_df(df_, sentences, flag, L, name):
    if flag == True:
        model = Word2Vec(sentences=sentences, size=L, window=10, min_count=2, workers=24, iter=10)
        model.save('output/model/' + name + '_w2c.model')
        print("模型保存完成！！！")
    model = Word2Vec.load('output/model/' + name + '_w2c.model')
    values=set(df_[name].values)
    w2v=[]

    for v in values:
        try:
            a=[int(v)]
            a.extend(model[str(v)])
            w2v.append(a)
        except:
            pass

    out_df=pd.DataFrame(w2v)
    del w2v
    gc.collect()
    out_df.columns = [name]+[name+'_embe_{}'.format(i) for i in range(L)]
#     embedding = pd.merge(df_,out_df,on=name,how='left')
#     del out_df
#     gc.collect()
    return out_df


In [14]:
out_df = get_embedding_df(train_test_features, sentences, True, 32, 'ad_id')
out_df

模型保存完成！！！


/opt/conda/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


,ad_id,ad_id_embe_0,ad_id_embe_1,ad_id_embe_2,ad_id_embe_3,ad_id_embe_4,ad_id_embe_5,ad_id_embe_6,ad_id_embe_7,ad_id_embe_8,...,ad_id_embe_22,ad_id_embe_23,ad_id_embe_24,ad_id_embe_25,ad_id_embe_26,ad_id_embe_27,ad_id_embe_28,ad_id_embe_29,ad_id_embe_30,ad_id_embe_31
0,1,0.105910,0.335036,-0.431351,0.164155,0.261301,0.634626,-0.009314,-0.023387,-0.264512,...,-0.251268,0.431521,0.835306,-0.021999,0.727791,-0.095861,0.132961,-0.257504,0.228655,0.597648
1,4,0.914675,0.297620,-0.213420,0.109848,0.114923,0.717790,-0.362810,-0.208374,-0.160634,...,0.291628,0.086271,0.197985,0.383805,0.575841,0.064396,0.247708,-0.360945,0.403407,0.914605
2,7,0.044670,0.167866,-0.237711,-0.043284,0.111577,-0.111510,-0.210162,-0.385787,-0.118634,...,0.017564,-0.104224,-0.031909,0.167522,0.066377,0.090214,0.057412,-0.134563,0.091735,0.194077
3,10,0.221213,0.166764,-0.256418,0.109516,0.002723,0.345446,-0.157145,-0.103459,-0.205685,...,0.055161,0.273621,0.314123,0.150131,0.421323,-0.113825,0.115182,-0.193191,0.076697,0.425385
4,13,0.529767,0.090097,-0.195150,-0.233836,0.060646,0.355769,0.134674,0.106647,-0.083118,...,0.138554,0.139170,0.086962,0.134809,0.176469,-0.101493,0.026703,-0.173130,0.188915,0.269375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1977172,3812192,0.137362,0.065485,-0.080351,0.005960,-0.034305,0.073356,-0.021868,-0.052291,-0.024175,...,-0.025813,0.033140,0.019157,0.070050,0.090867,0.026364,0.067118,-0.077113,0.050097,0.123414
1977173,3812193,0.173164,0.002196,-0.086958,-0.209613,0.291240,0.033864,-0.002423,-0.054711,0.093734,...,0.044948,0.103580,-0.056693,-0.182879,0.108853,-0.020049,-0.082090,-0.121302,-0.023137,0.093575
1977174,3812197,0.060151,-0.016913,0.011626,0.102653,0.002425,0.127144,-0.110589,-0.073005,-0.125047,...,0.080313,-0.010211,0.118214,0.039306,0.143943,-0.055003,0.066039,-0.108260,0.024133,0.107624
1977175,3812200,0.085246,0.014792,-0.049845,0.015642,0.016032,0.114203,-0.019417,-0.006446,-0.059986,...,0.015782,0.075696,0.057648,0.027031,0.129858,-0.033118,0.041175,-0.132730,0.021486,0.102729


In [15]:
out_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1977177 entries, 0 to 1977176
Data columns (total 33 columns):
 #   Column         Dtype  
---  ------         -----  
 0   ad_id          int64  
 1   ad_id_embe_0   float64
 2   ad_id_embe_1   float64
 3   ad_id_embe_2   float64
 4   ad_id_embe_3   float64
 5   ad_id_embe_4   float64
 6   ad_id_embe_5   float64
 7   ad_id_embe_6   float64
 8   ad_id_embe_7   float64
 9   ad_id_embe_8   float64
 10  ad_id_embe_9   float64
 11  ad_id_embe_10  float64
 12  ad_id_embe_11  float64
 13  ad_id_embe_12  float64
 14  ad_id_embe_13  float64
 15  ad_id_embe_14  float64
 16  ad_id_embe_15  float64
 17  ad_id_embe_16  float64
 18  ad_id_embe_17  float64
 19  ad_id_embe_18  float64
 20  ad_id_embe_19  float64
 21  ad_id_embe_20  float64
 22  ad_id_embe_21  float64
 23  ad_id_embe_22  float64
 24  ad_id_embe_23  float64
 25  ad_id_embe_24  float64
 26  ad_id_embe_25  float64
 27  ad_id_embe_26  float64
 28  ad_id_embe_27  float64
 29  ad_id_embe_28 

In [16]:
out_df.to_csv('output/model/ad_id_embedding.csv', index=False)